The applicable IAM role must include these permissions:
* ec2 run-instances
* ec2 describe-instances 
* ec2 terminate-instances

In [ ]:
%env REGION eu-west-1

# Deep Learning AMI in eu-west-1
%env AMI_ID ami-02273e0d16172dbd1

# Adapt these as needed
# The security must allow incoming TCP ports 22 (ssh) and 8888 (jupyter)
%env SECURITY_GROUP_ID sg-0262e9c0dad040106
%env ROLE_NAME AdminRole
%env KEY_NAME admin
%env KEY_PAIR ~/.ssh/admin.pem

In [ ]:
%%sh

aws configure set region $REGION

In [ ]:
%%sh

# Create a p3.2xlarge spot instance (1 NVIDIA V100 GPU)
aws ec2 run-instances --image-id $AMI_ID \
                       --instance-type p3.2xlarge \
                       --instance-market-options '{"MarketType":"spot"}' \
                       --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=dlami-demo}]' \
                       --key-name $KEY_NAME \
                       --security-group-ids $SECURITY_GROUP_ID \
                       --iam-instance-profile Name=$ROLE_NAME

In [ ]:
%%sh

# Get the public DNS name
export INSTANCE_NAME=`aws ec2 describe-instances \
     --filters "Name=tag:Name,Values=dlami-demo" "Name=instance-state-name,Values=running" \
     --query "Reservations[*].Instances[*].PublicDnsName" --output text`

echo $INSTANCE_NAME

In [ ]:
%%sh 

# Get the instance id
export INSTANCE_ID=`aws ec2 describe-instances \
     --filters "Name=tag:Name,Values=dlami-demo" "Name=instance-state-name,Values=running" \
     --query "Reservations[*].Instances[*].InstanceId" --output text`

echo $INSTANCE_ID

# Terminate the instance
aws ec2 terminate-instances --instance-ids $INSTANCE_ID